In [4]:
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_rows', None)

# input an URL and return a generator of following page URL

In [5]:
def geturl(url, page): 
    res = requests.get(url,cookies={'over18': '1'})
    soup = BeautifulSoup(res.text,'html.parser')
    btn = soup.find_all(class_="btn wide")
    page_code = int(re.search('\d{5}',btn[1]['href']).group())+1


    for i in range(page):
        urls = "https://www.ptt.cc/bbs/Gossiping/index" + str(page_code - i) + ".html"
        yield urls

# Receice an URL list ot generator and get useful data cell

In [6]:
#---------------------------------------------------------------------------------------------------
def get_data(urls):
    

    for url_new in urls:
        time.sleep(0.5)
        res = requests.get(url_new,cookies={'over18': '1'})
        soup = BeautifulSoup(res.text,'html.parser')
        soup_div_all = soup.find_all('div',class_="r-ent")

        yield soup_div_all


#  split data to what we need

In [7]:
def filter_data(soup_div_all):
    # soup_div[0]
    
    title = []
    date = []
    author = []
    push = []
    link = []
    
    for soup_div in soup_div_all:
        length = len(soup_div)
        for i in range(length,-1,-1):
            try:
                title.append(soup_div[i].find('a').text)
                date.append(soup_div[i].find(class_="date").text.strip())

                author.append(soup_div[i].find(class_='author').text)
                link.append(soup_div[i].find('a')['href'])


                try:
                    push_old = soup_div[i].find('span').text
                    
                    if re.search('\A\d+', push_old) != None:
                        push.append(int(push_old))
                    
                    elif re.search('X+\d?|爆',push_old) != None:  
                        word_new = word_to_int(push_old)
                        push.append(word_new)
                    else:
                        push.append('?????????????')
    #                 except :
                        print(push_old)
                except:
                    push.append(0)

            except:
                continue
    dict_new = {'date' : date, 'title' : title, 'author' : author, 'push' : push, 'link' : link}
    print(len(push))
    print(len(title))
    return dict_new

# convert push to int

In [8]:
def word_to_int(word):
    try:
        word_new = -1 * int(word[1])
    except:
        word_new = 100
    return word_new

In [12]:
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
page = 10

# for i in geturl(url):
#     # print(i)

#     filter_data(i)
#     time.sleep(2)

soup_div_all = get_data(geturl(url, page))
dict_new = filter_data(soup_div_all)

194
194


In [13]:
new_df = pd.DataFrame(dict_new)
new_df
new_df.sort_values(by=['push'])

,date,title,author,push,link
185,4/05,[新聞] 總統：太魯閣號事故賠償支出 不會來自愛,kuro,-1,/bbs/Gossiping/M.1617633056.A.19A.html
36,4/05,[問卦] 有人連假都在ptt噓人嗎？,pupss89177,-1,/bbs/Gossiping/M.1617638283.A.BF9.html
135,4/05,[問卦] 反制龜速車最好的辦法,facebookig,0,/bbs/Gossiping/M.1617634615.A.277.html
122,4/05,Re: [問卦] 為何台灣早就知道半導體晶片的重要性 ?,Sylph,0,/bbs/Gossiping/M.1617635179.A.A02.html
119,4/05,[問卦] 覺得朋友開的地獄梗很好笑要怎麼辦？,k99t,0,/bbs/Gossiping/M.1617635298.A.138.html
31,4/05,Re: [新聞] 太魯閣號51死 新北巿府成立捐款專戶指定,tingdou,0,/bbs/Gossiping/M.1617638348.A.C02.html
118,4/05,[問卦] 高雄氣爆裝置藝術和出國推廣與善款？,r5e97nk63,0,/bbs/Gossiping/M.1617635333.A.6F3.html
116,4/05,Re: [問卦] 死掉的網紅根本不紅 為什麼媒體稱網紅？,Paul1021,0,/bbs/Gossiping/M.1617635352.A.C35.html
34,4/05,[問卦] 有沒有人知道這是什麼EVA裝置藝術?,ezJapan,0,/bbs/Gossiping/M.1617638308.A.573.html
102,4/05,Re: [新聞] 台鐵出軌 日媒：暴露台灣基礎設施缺陷,Sylph,0,/bbs/Gossiping/M.1617635850.A.D08.html


In [14]:
new_df


,date,title,author,push,link
0,4/06,[問卦] 國動在這次出軌事件有沒有發表什麼？,ann141414,1,/bbs/Gossiping/M.1617639183.A.088.html
1,4/06,[問卦] 我永遠 都想不到陪我看這書的妳會要走,HDSM,0,/bbs/Gossiping/M.1617639160.A.8D9.html
2,4/06,Re: [新聞] 台鐵出軌 日媒：暴露台灣基礎設施缺陷,bigbo,0,/bbs/Gossiping/M.1617639151.A.A7F.html
3,4/06,[問卦] 每到午夜零時就活力十足的是什麼人?,cwh0105,1,/bbs/Gossiping/M.1617639139.A.ECD.html
4,4/06,[問卦] 484很多人不知道重力加速度是什麼,soulXdancer,4,/bbs/Gossiping/M.1617639075.A.4D9.html
5,4/06,Re: [新聞] 柯文哲赴女士官長靈前上香 不捨市府司機,kid725,1,/bbs/Gossiping/M.1617639058.A.2AC.html
6,4/06,[問卦] 給賈伯斯經營台鐵有救嗎?,lapdada,0,/bbs/Gossiping/M.1617639045.A.2B2.html
7,4/06,[問卦] 台中人現在停水在做甚麼,soi1dsnake,0,/bbs/Gossiping/M.1617639024.A.1C6.html
8,4/06,Re: [問卦] 有沒有可能在台北買地自建,gerychen,0,/bbs/Gossiping/M.1617638995.A.9E4.html
9,4/06,[問卦] 太魯閣事件捐款的意義？,miniuser,0,/bbs/Gossiping/M.1617638962.A.5C0.html
